# My Pure Pix2Pix Implementation

****



## General Presets

In [11]:
""" Import Packages """

# Numerical Operations
import random
import numpy as np
# Reading/Writing/Cleaning Data
import pandas as pd
from PIL import Image
import os
import gc
# For Progress Bar
from tqdm.auto import tqdm
# For Display
import matplotlib.pyplot as plt
import wandb as wb
# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset, random_split
from torchvision.datasets import DatasetFolder, VisionDataset
# my_utilities
import my_utilities


In [12]:
""" For Display """

plt.style.use(r'C:\Users\chael\.matplotlib\violet.mplstyle')
wb.init(project="pure_pix2pix", name="run0")


In [13]:
""" For Auto-reload Modules"""

%reload_ext autoreload
%autoreload 2

In [14]:
""" Assure Reproducibility """

my_utilities.assure_reproduce(29)

## Configurations

In [15]:
""" Path & Device"""

data_dir='datasets/facades'
device=my_utilities.register_device()